# GRU Classification with MPQA Dataset
<hr>

We will build a text classification model using GRU model on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


,sentence,label,split
0,complaining,0,train
1,failing to support,0,train
2,desperately needs,0,train
3,many years of decay,0,train
4,no quick fix,0,train
...,...,...,...
10601,urged,1,train
10602,strictly abide,1,train
10603,hope,1,train
10604,strictly abide,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,7294,7294
1,3312,3312


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## GRU Model

In [17]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        # tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [18]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 128)               74496     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 515,169
Trainable params: 515,169
Non-trainable params: 0
_________________________________________________________________


In [19]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [20]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=30, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/30
299/299 [==============================] - 26s 48ms/step - loss: 0.5407 - accuracy: 0.7347 - val_loss: 0.3778 - val_accuracy: 0.8445
Epoch 2/30
299/299 [==============================] - 10s 35ms/step - loss: 0.1923 - accuracy: 0.9330 - val_loss: 0.3706 - val_accuracy: 0.8407
Epoch 3/30
299/299 [==============================] - 10s 35ms/step - loss: 0.1330 - accuracy: 0.9545 - val_loss: 0.4169 - val_accuracy: 0.8520
Epoch 4/30
299/299 [==============================] - 10s 35ms/step - loss: 0.1046 - accuracy: 0.9634 - val_loss: 0.4712 - val_accuracy: 0.8388
Epoch 5/30
299/299 [==============================] - 11s 35ms/step - loss: 0.0761 - accuracy: 0.9697 - val_loss: 0.4566 - val_accuracy: 0.8539
Epoch 6/30
299/299 [==============================] - 10s 35ms/step - loss: 0.0585 - accuracy: 0.9761 - val_loss: 0.5242 - val_accuracy: 0.8520
Epoch 7/30
299/299 [==============================] - 10s 35ms/step - loss: 0.0466 - accuracy: 0.9824 - val_loss: 0.5278 - 

## Summary

In [21]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,85.956645,87.181902,84.731388,86.33365,85.862392,86.899149,85.283017,86.698115,84.622639,85.094339,85.866324


In [22]:
report = record
report = report.to_excel('GRU_MPQA_v2.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## GRU Model

In [18]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        # tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [19]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               74496     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 515,169
Trainable params: 215,169
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [21]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 45s 94ms/step - loss: 0.4234 - accuracy: 0.8070 - val_loss: 0.2990 - val_accuracy: 0.8784
Epoch 2/40
299/299 [==============================] - 20s 68ms/step - loss: 0.2867 - accuracy: 0.8887 - val_loss: 0.2983 - val_accuracy: 0.8794
Epoch 3/40
299/299 [==============================] - 20s 68ms/step - loss: 0.2485 - accuracy: 0.9063 - val_loss: 0.3052 - val_accuracy: 0.8728
Epoch 4/40
299/299 [==============================] - 20s 67ms/step - loss: 0.2158 - accuracy: 0.9165 - val_loss: 0.3022 - val_accuracy: 0.8831
Epoch 5/40
299/299 [==============================] - 20s 69ms/step - loss: 0.2029 - accuracy: 0.9195 - val_loss: 0.3212 - val_accuracy: 0.8737
Epoch 6/40
299/299 [==============================] - 20s 68ms/step - loss: 0.1763 - accuracy: 0.9311 - val_loss: 0.3412 - val_accuracy: 0.8586
Epoch 7/40
299/299 [==============================] - 20s 67ms/step - loss: 0.1464 - accuracy: 0.9460 - val_loss: 0.3483 - 

Epoch 16/40
299/299 [==============================] - 40s 134ms/step - loss: 0.0550 - accuracy: 0.9793 - val_loss: 0.7080 - val_accuracy: 0.8728
Epoch 17/40
299/299 [==============================] - 39s 131ms/step - loss: 0.0558 - accuracy: 0.9811 - val_loss: 0.7061 - val_accuracy: 0.8662
Epoch 18/40
299/299 [==============================] - 38s 127ms/step - loss: 0.0527 - accuracy: 0.9795 - val_loss: 0.6891 - val_accuracy: 0.8671
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Test Accuracy: 87.74740695953369
Training 5: 
Epoch 1/40
299/299 [==============================] - 53s 120ms/step - loss: 0.4205 - accuracy: 0.8094 - val_loss: 0.3394 - val_accuracy: 0.8586
Epoch 2/40
299/299 [==============================] - 24s 82ms/step - loss: 0.2688 - accuracy: 0.8936 - val_loss: 0.3349 - val_accuracy: 0.8652
Epoch 3/40
299/299 [==============================] - 25s 85ms/step - loss: 0.2411 - accuracy: 0.9060 - val_loss: 0.3425 - val_accuracy: 0.8633

Epoch 10/40
299/299 [==============================] - 50s 167ms/step - loss: 0.0963 - accuracy: 0.9636 - val_loss: 0.4624 - val_accuracy: 0.8632
Epoch 11/40
299/299 [==============================] - 49s 164ms/step - loss: 0.0821 - accuracy: 0.9709 - val_loss: 0.4649 - val_accuracy: 0.8679
Epoch 12/40
299/299 [==============================] - 49s 163ms/step - loss: 0.0652 - accuracy: 0.9761 - val_loss: 0.5437 - val_accuracy: 0.8585
Epoch 13/40
299/299 [==============================] - 49s 163ms/step - loss: 0.0652 - accuracy: 0.9752 - val_loss: 0.5398 - val_accuracy: 0.8717
Epoch 14/40
299/299 [==============================] - 49s 163ms/step - loss: 0.0617 - accuracy: 0.9769 - val_loss: 0.5268 - val_accuracy: 0.8717
Epoch 15/40
299/299 [==============================] - 49s 163ms/step - loss: 0.0675 - accuracy: 0.9763 - val_loss: 0.5674 - val_accuracy: 0.8708
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 87.16981410980225
Trainin

## Summary

In [22]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,88.312912,88.689917,89.443922,87.747407,87.65316,89.349669,89.33962,87.169814,86.698115,86.320752,88.072529


In [23]:
report = record2
report = report.to_excel('GRU_MPQA_v2_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## GRU Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)),
        # tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 100, 128)          140544    
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 128)               74496     
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 515,169
Trainable params: 515,169
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [28]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=6, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=40, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 
Epoch 1/40
299/299 [==============================] - 112s 310ms/step - loss: 0.4151 - accuracy: 0.8125 - val_loss: 0.3366 - val_accuracy: 0.8643
Epoch 2/40
299/299 [==============================] - 69s 231ms/step - loss: 0.1785 - accuracy: 0.9357 - val_loss: 0.3543 - val_accuracy: 0.8633
Epoch 3/40
299/299 [==============================] - 79s 263ms/step - loss: 0.1214 - accuracy: 0.9584 - val_loss: 0.4109 - val_accuracy: 0.8558
Epoch 4/40
299/299 [==============================] - 79s 263ms/step - loss: 0.0865 - accuracy: 0.9686 - val_loss: 0.4483 - val_accuracy: 0.8549
Epoch 5/40
299/299 [==============================] - 83s 277ms/step - loss: 0.0660 - accuracy: 0.9761 - val_loss: 0.5224 - val_accuracy: 0.8520
Epoch 6/40
299/299 [==============================] - 82s 273ms/step - loss: 0.0464 - accuracy: 0.9813 - val_loss: 0.5542 - val_accuracy: 0.8492
Epoch 7/40
299/299 [==============================] - 81s 271ms/step - loss: 0.0438 - accuracy: 0.9843 - val_loss: 0

Epoch 6/40
299/299 [==============================] - 88s 293ms/step - loss: 0.0514 - accuracy: 0.9776 - val_loss: 0.5102 - val_accuracy: 0.8642
Epoch 7/40
299/299 [==============================] - 89s 297ms/step - loss: 0.0418 - accuracy: 0.9826 - val_loss: 0.5776 - val_accuracy: 0.8660
Epoch 8/40
299/299 [==============================] - 88s 293ms/step - loss: 0.0385 - accuracy: 0.9821 - val_loss: 0.6697 - val_accuracy: 0.8604
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 88.96226286888123
Training 8: 
Epoch 1/40
299/299 [==============================] - 145s 423ms/step - loss: 0.3995 - accuracy: 0.8216 - val_loss: 0.3483 - val_accuracy: 0.8613
Epoch 2/40
299/299 [==============================] - 122s 409ms/step - loss: 0.1835 - accuracy: 0.9335 - val_loss: 0.4184 - val_accuracy: 0.8613
Epoch 3/40
299/299 [==============================] - 121s 406ms/step - loss: 0.1180 - accuracy: 0.9561 - val_loss: 0.3949 - val_accuracy: 0.85

## Summary

In [30]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,86.427897,87.935907,88.124412,88.784164,89.066917,88.218659,88.962263,86.132073,88.01887,88.773584,88.044475


In [31]:
report = record3
report = report.to_excel('GRU_MPQA_v2_3.xlsx', sheet_name='dynamic')